# Multivariate statistics (decoding / MVPA) on MEG/EEG

Author : Alexandre Gramfort

See more info on decoding on this page: https://mne.tools/stable/auto_tutorials/machine-learning/plot_sensors_decoding.html

In [1]:
# add plot inline in the page
%matplotlib qt
import os
import matplotlib.pyplot as plt

First, load the mne package:

In [2]:
import mne

We set the log-level to 'WARNING' so the output is less verbose

In [3]:
mne.set_log_level('WARNING')

## Access raw data

Now we import the sample dataset. If you don't already have it, it will be downloaded automatically (but be patient approx. 2GB)

In [4]:
mne.set_log_level('WARNING')

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("/mnt/c/Users/Egor/Downloads/MEEG/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')

## Read and crop epochs

In [5]:
epochs = mne.read_epochs(epochs_fname)
epochs.crop(None, 0.25)

<EpochsFIF  |   140 events (all good), -0.5 - 0.25 sec, baseline [-0.2, 0], ~98.9 MB, data loaded,
 'face/famous/first': 24
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 24
 'face/unfamiliar/immediate': 9
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 13
 'scrambled/long': 11>

Look at the ERF and contrast between left and rigth response

In [6]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()
evoked_contrast = mne.combine_evoked([evoked_face, evoked_scrambled],
                                     [0.5, -0.5])

In [7]:
fig = evoked_face.plot()
fig = evoked_scrambled.plot()
fig = evoked_contrast.plot()

Plot some topographies

In [8]:
vmin, vmax = -4, 4
fig = evoked_face.plot_topomap(ch_type='eeg', contours=0, vmin=vmin, vmax=vmax)
fig = evoked_scrambled.plot_topomap(ch_type='eeg', contours=0, vmin=vmin, vmax=vmax)
fig = evoked_contrast.plot_topomap(ch_type='eeg', contours=0, vmin=None, vmax=None)

## Now let's see if we can classify single trials

To have a chance at 50% accuracy equalize epoch count in each condition

In [9]:
epochs.equalize_event_counts(['face', 'scrambled'])

(<EpochsFIF  |   98 events (all good), -0.5 - 0.25 sec, baseline [-0.2, 0], ~71.5 MB, data loaded,
  'face/famous/first': 11
  'face/famous/immediate': 8
  'face/famous/long': 10
  'face/unfamiliar/first': 10
  'face/unfamiliar/immediate': 6
  'face/unfamiliar/long': 4
  'scrambled/first': 25
  'scrambled/immediate': 13
  'scrambled/long': 11>,
 array([  0,   1,   2,   3,   4,   5,   6,   9,  10,  11,  12,  23,  26,
         27,  28,  57,  58,  63,  64,  65,  68,  69,  90, 105, 106, 107,
        110, 117, 118, 119, 120, 121, 122, 123, 132, 133, 134, 135, 136,
        137, 138, 139]))

In [10]:
print(epochs)

<EpochsFIF  |   98 events (all good), -0.5 - 0.25 sec, baseline [-0.2, 0], ~71.5 MB, data loaded,
 'face/famous/first': 11
 'face/famous/immediate': 8
 'face/famous/long': 10
 'face/unfamiliar/first': 10
 'face/unfamiliar/immediate': 6
 'face/unfamiliar/long': 4
 'scrambled/first': 25
 'scrambled/immediate': 13
 'scrambled/long': 11>


A classifier takes as input an `x` and return `y` (0 or 1). Here x will be the data at one time point on all gradiometers (hence the term multivariate). We work with all sensors jointly and try to find a discriminative pattern between 2 conditions to predict the class.

For classification we will use the scikit-learn package (http://scikit-learn.org/) and MNE functions 

`
Reference:
Scikit-learn: Machine Learning in Python,
Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
`

In [11]:
epochs.event_id

{'face/famous/first': 5,
 'face/famous/immediate': 6,
 'face/famous/long': 7,
 'face/unfamiliar/first': 13,
 'face/unfamiliar/immediate': 14,
 'face/unfamiliar/long': 15,
 'scrambled/first': 17,
 'scrambled/immediate': 18,
 'scrambled/long': 19}

In [12]:
import numpy as np
# make response vector
y = np.zeros(len(epochs.events), dtype=int)
y[epochs.events[:, 2] < 17] = 1  # 1 means face

y.size

98

In [13]:
X = epochs.copy().pick_types(meg='grad').get_data()
X.shape

(98, 204, 226)

In [15]:
XX = X.reshape(y.size, -1)
#XX = X.reshape(60, -1)
XX.shape

(98, 46104)

In [16]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1e6, solver='liblinear')
cv = StratifiedKFold(n_splits=5, random_state=42)
scores = cross_val_score(logreg, XX, y, cv=cv, scoring='roc_auc')
print(scores)
print('Accuracy = %0.3f (std %.3f)' % (np.mean(scores), np.std(scores)))

[0.84       0.7        0.77       0.69       0.74074074]
Accuracy = 0.748 (std 0.054)


In [17]:
plt.hist(scores, bins=20)

(array([1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.]),
 array([0.69  , 0.6975, 0.705 , 0.7125, 0.72  , 0.7275, 0.735 , 0.7425,
        0.75  , 0.7575, 0.765 , 0.7725, 0.78  , 0.7875, 0.795 , 0.8025,
        0.81  , 0.8175, 0.825 , 0.8325, 0.84  ]),
 <a list of 20 Patch objects>)

Now we can do this more simply using the `mne.decoding` module

In [18]:
from sklearn.pipeline import make_pipeline
from mne.decoding import Scaler, Vectorizer, cross_val_multiscore

epochs_decoding = epochs.copy().pick_types(meg='grad')

clf = make_pipeline(Scaler(epochs_decoding.info),
                    Vectorizer(),
                    logreg)

X = epochs_decoding.get_data()

scores = cross_val_multiscore(clf, X, y, cv=5, n_jobs=1)

# Mean scores across cross-validation splits
score = np.mean(scores, axis=0)
print('Spatio-temporal: %0.1f%%' % (100 * score,))

Spatio-temporal: 74.4%


## Decoding over time

In [20]:
from sklearn.preprocessing import StandardScaler
from mne.decoding import SlidingEstimator

clf = make_pipeline(StandardScaler(), logreg)

time_decod = SlidingEstimator(clf, n_jobs=1, scoring='roc_auc', verbose=True)
scores = cross_val_multiscore(time_decod, X, y, cv=5, n_jobs=1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(epochs.times, scores, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

[............................................................] 100.00% Fitting SlidingEstimator |
[............................................................] 100.00% Fitting SlidingEstimator |
[............................................................] 100.00% Fitting SlidingEstimator |
[............................................................] 100.00% Fitting SlidingEstimator |
[............................................................] 100.00% Fitting SlidingEstimator |


Text(0.5, 1.0, 'Sensor space decoding')

For more details see: https://mne.tools/stable/auto_tutorials/machine-learning/plot_sensors_decoding.html

and this book chapter:

Jean-Rémi King, Laura Gwilliams, Chris Holdgraf, Jona Sassenhagen, Alexandre Barachant, Denis Engemann, Eric Larson, Alexandre Gramfort. Encoding and Decoding Neuronal Dynamics: Methodological Framework to Uncover the Algorithms of Cognition. 2018. https://hal.archives-ouvertes.fr/hal-01848442/

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Do a generalization over time analysis as explained in the <a href="https://mne.tools/stable/auto_tutorials/machine-learning/plot_sensors_decoding.html">documentation on decoding</a>.</li>
    </ul>
</div>